## Importing all necessary libararies

In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd
from Utility.ThesisUtility import getSentcesAndTokens,iterateAndAssignGoldStandard

### Constants

In [ ]:
#The file path to the dataset
dataFilePath = r"D:\UDE\6th Semester\MEMS\MEWS Data\MEWS_Essays\MEWS_Essays\Essays_all\TelevisonMergedT1+T2"

### Reading the input(Essays) as text files and parsing it into sentences and tokens

In [2]:
sentences = getSentcesAndTokens(dataFilePath,False)

## Reading the gold standards and vectorizing them
df_ref = pd.read_excel(r"D:\USERS-Load\PycharmProjects\pythonProject\CSV Files\Ads prompt\01.08.22\new Gold standards.xlsx")
listRef = df_ref['Gold standard']

### building the sentences' vectors using TF-IDF vectorizer

In [6]:
vectorizer = TfidfVectorizer(min_df=1, stop_words='english')
vectorizer.fit_transform(sentences)
vectors = vectorizer.transform(sentences)

refs = vectorizer.transform(listRef)

### Iterating over all the sentneces and assigning the gold standard with the highest cosin similarity

In [ ]:
iterateAndAssignGoldStandard(vectors,sentences,refs)